In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
ds = pd.read_csv('./titanic.csv')
print (ds.columns)
print (len(ds.columns))
ds.head()

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
12


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
cols_to_remove = [
    'PassengerId',
    'Name',
    'Cabin',
    'Embarked',
    'Ticket'
]
df = ds.drop(cols_to_remove, axis=1)
print (len(df.columns))

df.head()

7


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [4]:
sex_mapping = {
    'male': 0,
    'female': 1,
}
df.Sex = df.Sex.map(sex_mapping)

In [5]:
data = df.values
print (data.shape)

df = df.dropna(axis=0)

data = df.values
print (data.shape)

(891, 7)
(714, 7)


In [6]:
X = data[:,1:] ## input
Y = data[:, 0] ## labels

print (X.shape)
print (Y.shape)

split = int(0.8*X.shape[0])

X_train = X[:split]
Y_train = Y[:split]

X_test = X[split:]
Y_test = Y[split:]

print ("-----------")

print (X_train.shape)
print (X_test.shape)

print (Y_train.shape)
print (Y_test.shape)

(714, 6)
(714,)
-----------
(571, 6)
(143, 6)
(571,)
(143,)


In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
selector=SelectKBest(chi2,k=4)
X_new=selector.fit_transform(X_train,Y_train)
print(X_new.shape)
scores=selector.scores_
print(scores)
print (scores.shape)

(571, 4)
[2.22660360e+01 1.01895154e+02 2.76308060e+01 8.65675925e-01
 3.50027893e+00 2.67625670e+03]
(6,)


In [8]:
# implementing logistic regression in titani data set


from sklearn.linear_model import LogisticRegression

In [9]:
lr = LogisticRegression()
lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
lr.score(X_test, Y_test)

#end

0.8181818181818182

In [11]:
import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [12]:
dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)
print (dt.score(X_test, Y_test))

## Random forest
rf = RandomForestClassifier(n_estimators=145)
rf.fit(X_train, Y_train)
print (rf.score(X_test, Y_test))

0.7972027972027972
0.8531468531468531


In [13]:
### Our decision tree
print data.shape

def data_split(dataset, col, value):
    
    data_right = []
    data_left = []
    
    for ix in range(dataset.shape[0]):
        
        if dataset[ix, col] > value:
            data_right.append(data[ix,:])
        else:
            data_left.append(data[ix, :])
    
    return np.array(data_right), np.array(data_left)

def entropy(dataset, col=0):
    p = []
    
    p_survive = dataset[:, col].mean()
    p.append(p_survive)
    
    p.append(1-p[0])
    
    ent = 0.0
    for px in p:
        ent += (-1.0*px*np.log2(px))
    
    return ent

def information_gain(dataset,subset):
    return entropy(dataset) - entropy(subset)

def information_gain2(dataset, left_subset, right_subset):
    
    left = left_subset
    right = right_subset
    
    h_data = entropy(dataset)
    if left.shape[0]>0:
        left_gain = h_data - entropy(left)
    else:
        left_gain = -100000
        
    if right.shape[0]>0:
        right_gain = h_data - entropy(right)
    else:
        right_gain = -100000
    
    total_gain = right_gain + left_gain
    return total_gain

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(data.shape)? (<ipython-input-13-665635cd13c0>, line 2)

In [ ]:
INFINITY = -10000000
class DT:
    def __init__(self, depth, max_depth):
        self.right = None
        self.left = None
        self.col_id = None
        self.value = None
        self.depth = depth
        self.max_depth = max_depth
    
    def select_attr_And_run(self, dataset):
        
        ## Exit Condition
        if self.depth>= self.max_depth:
            return 
        
        n_cols = 6
        start_index = 1
        check_index = 0
        
        all_gains = []
        
        for cx in range(start_index, start_index+n_cols):
            split_val = dataset[:, cx].mean()
            
            right, left = data_split(dataset, cx, split_val)
            
            if left.shape[0] > 0:
                left_gain = information_gain(dataset, left)
            else:
                left_gain = INFINITY
            
            if right.shape[0] > 0:
                right_gain = information_gain(dataset, right)
            else:
                right_gain = INFINITY
            
            comb_gain = right_gain + left_gain
            all_gains.append(comb_gain)
        
        all_gains = np.array(all_gains)
        self.col_id = np.argmax(all_gains) + start_index
        self.value = dataset[:, self.col_id].mean()
        
        data_right, data_left = data_split(dataset, self.col_id, self.value)
        
        if data_right.shape[0] > 0:
            self.right = DT(self.depth+1,self.max_depth)
            self.right.select_attr_And_run(data_right)
        
        if data_left.shape[0] > 0:
            self.left = DT(self.depth+1, self.max_depth)
            self.left.select_attr_And_run(data_left)
        
        return 
    
    def predict(example):
        ## example of shape :- (1, #features)
        ## Here -> (1,6)
        pass

In [ ]:
print (data.shape)
dt = DT(depth=0, max_depth=5)
dt.select_attr_And_run(data)
# dt.select_attr_And_run(da)